In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [ ]:
players <- read_csv("players.csv")
players

players_select <- players |>
    select(experience, played_hours, Age)
players_select

players_split <- initial_split(players_select, prop = 0.75, strata = played_hours)
players_training <- training(players_split)
players_training
players_testing <- testing(players_split)
players_testing

In [ ]:
players_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
    set_engine("kknn") |>
    set_mode("regression") 

players_recipe <- recipe(played_hours ~ ., data = players_training) |>
    step_dummy(all_nominal_predictors(), -all_outcomes()) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

players_vfold <- vfold_cv(players_training, v = 5, strata = played_hours)

players_workflow <- workflow() |> 
    add_recipe(players_recipe) |> 
    add_model(players_spec)
players_workflow

In [ ]:
gridvals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

players_results <- workflow() |> 
    add_recipe(players_recipe) |> 
    add_model(players_spec) |>
    tune_grid(object = players_workflow, resamples = players_vfold, grid = gridvals) |>
    collect_metrics()
players_results

players_min <- players_results |>
    filter(.metric == "rmse") |>
    slice_min(std_err, n = 1)
players_min